In [1]:
%matplotlib inline
import os
import collections
import torch
import torchvision
import numpy as np
import scipy.misc as m
import scipy.io as io
import matplotlib.pyplot as plt

from SUNRGBDLoader import *

os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
import sys, os
import torch
import visdom
import argparse
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

from torch.autograd import Variable
from torch.utils import data
from tqdm import tqdm

from ptsemseg.models import get_model
from ptsemseg.loader import get_loader, get_data_path
from ptsemseg.metrics import runningScore
from ptsemseg.loss import *
from ptsemseg.augmentations import *


parser = argparse.ArgumentParser(description='Hyperparams')
parser.add_argument('--arch', nargs='?', type=str, default='fcn8s', 
                    help='Architecture to use [\'fcn8s, unet, segnet etc\']')
parser.add_argument('--img_rows', nargs='?', type=int, default=256, 
                    help='Height of the input image')
parser.add_argument('--img_cols', nargs='?', type=int, default=256, 
                    help='Width of the input image')

parser.add_argument('--img_norm', dest='img_norm', action='store_true', 
                    help='Enable input image scales normalization [0, 1] | True by default')
parser.add_argument('--no-img_norm', dest='img_norm', action='store_false', 
                    help='Disable input image scales normalization [0, 1] | True by default')
parser.set_defaults(img_norm=True)

parser.add_argument('--n_epoch', nargs='?', type=int, default=100, 
                    help='# of the epochs')
parser.add_argument('--batch_size', nargs='?', type=int, default=1, 
                    help='Batch Size')
parser.add_argument('--l_rate', nargs='?', type=float, default=1e-5, 
                    help='Learning Rate')
parser.add_argument('--feature_scale', nargs='?', type=int, default=1, 
                    help='Divider for # of features to use')
parser.add_argument('--resume', nargs='?', type=str, default=None,    
                    help='Path to previous saved model to restart from')

parser.add_argument('--visdom', dest='visdom', action='store_true', 
                    help='Enable visualization(s) on visdom | False by default')
parser.add_argument('--no-visdom', dest='visdom', action='store_false', 
                    help='Disable visualization(s) on visdom | False by default')
parser.set_defaults(visdom=False)


In [ ]:
args = parser.parse_args('--arch fcn8s_rgbd_renet --batch_size 3'.split(' '))

# Setup Augmentations
data_aug= Compose([RandomRotate(10),                                        
                   RandomHorizontallyFlip()])

# Setup Dataloader
data_path = '/home/dongwonshin/Desktop/Datasets/SUNRGBD/SUNRGBD(renamed)/'
t_loader = SUNRGBDLoader(data_path, is_transform=True)
v_loader = SUNRGBDLoader(data_path, is_transform=True, split='val')

n_classes = t_loader.n_classes
trainloader = data.DataLoader(t_loader, batch_size=args.batch_size, num_workers=16, shuffle=True)
valloader = data.DataLoader(v_loader, batch_size=args.batch_size, num_workers=16)

# Setup Metrics
running_metrics = runningScore(n_classes)

# Setup visdom for visualization
if args.visdom:
    vis = visdom.Visdom()

    loss_window = vis.line(X=torch.zeros((1,)).cpu(),
                       Y=torch.zeros((1)).cpu(),
                       opts=dict(xlabel='minibatches',
                                 ylabel='Loss',
                                 title='Training Loss',
                                 legend=['Loss']))

# Setup Model
model = get_model(args.arch, n_classes)

model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
model.cuda()

# Check if model has custom optimizer / loss
if hasattr(model.module, 'optimizer'):
    optimizer = model.module.optimizer
else:
    optimizer = torch.optim.SGD(model.parameters(), lr=args.l_rate, momentum=0.99, weight_decay=5e-4)

if hasattr(model.module, 'loss'):
    print('Using custom loss')
    loss_fn = model.module.loss
else:
    loss_fn = cross_entropy2d

if args.resume is not None:                                         
    if os.path.isfile(args.resume):
        print("Loading model and optimizer from checkpoint '{}'".format(args.resume))
        checkpoint = torch.load(args.resume)
        model.load_state_dict(checkpoint['model_state'])
        optimizer.load_state_dict(checkpoint['optimizer_state'])
        print("Loaded checkpoint '{}' (epoch {})"                    
              .format(args.resume, checkpoint['epoch']))
    else:
        print("No checkpoint found at '{}'".format(args.resume)) 


# Training

In [ ]:
best_iou = -100.0 
for epoch in range(args.n_epoch):
    model.train()
    for i, (color_imgs, depth_imgs, label_imgs) in enumerate(trainloader):
        images = Variable(color_imgs.cuda())
        depth_images = Variable(depth_imgs.cuda())
        labels = Variable(label_imgs.cuda())

        optimizer.zero_grad()
        outputs = model(images,depth_images)

        loss = loss_fn(input=outputs, target=labels)

        loss.backward()
        optimizer.step()

        if args.visdom:
            vis.line(
                X=torch.ones((1, 1)).cpu() * i,
                Y=torch.Tensor([loss.data[0]]).unsqueeze(0).cpu(),
                win=loss_window,
                update='append')

        if (i+1) % 20 == 0:
            print("Epoch [%d/%d] Loss: %.4f" % (epoch+1, args.n_epoch, loss.data[0]))

    model.eval()
    for i_val, (color_images_val, depth_images_val, label_images_val) in tqdm(enumerate(valloader)):
        color_images_val = Variable(color_images_val.cuda(), volatile=True)
        depth_images_val = Variable(depth_images_val.cuda(), volatile=True)
        label_images_val = Variable(label_images_val.cuda(), volatile=True)

        outputs = model(color_images_val, depth_images_val)
        pred = outputs.data.max(1)[1].cpu().numpy()
        gt = label_images_val.data.cpu().numpy()
        running_metrics.update(gt, pred)

    score, class_iou = running_metrics.get_scores()
    for k, v in score.items():
        print(k, v)
    running_metrics.reset()

    if score['Mean IoU : \t'] >= best_iou:
        best_iou = score['Mean IoU : \t']
        state = {'epoch': epoch+1,
                 'model_state': model.state_dict(),
                 'optimizer_state' : optimizer.state_dict(),}
        torch.save(state, "{}_{}_best_model.pkl".format(args.arch, 'SUNRGBD'))

/home/dongwonshin/.conda/envs/pytorch/lib/python3.6/site-packages/torch/nn/functional.py:1462: UserWarning: nn.functional.upsample_bilinear is deprecated. Use nn.functional.upsample instead.
  warnings.warn("nn.functional.upsample_bilinear is deprecated. Use nn.functional.upsample instead.")


Epoch [1/100] Loss: 3.6067
Epoch [1/100] Loss: 3.3108
Epoch [1/100] Loss: 2.9417
Epoch [1/100] Loss: 2.4377
Epoch [1/100] Loss: 2.5109
Epoch [1/100] Loss: 2.3455
Epoch [1/100] Loss: 2.3839
Epoch [1/100] Loss: 2.0443
Epoch [1/100] Loss: 2.3408
Epoch [1/100] Loss: 1.9038
Epoch [1/100] Loss: 2.3707
Epoch [1/100] Loss: 1.8259
Epoch [1/100] Loss: 3.1192
Epoch [1/100] Loss: 1.7552
Epoch [1/100] Loss: 2.0447
Epoch [1/100] Loss: 2.0158
Epoch [1/100] Loss: 1.8998
Epoch [1/100] Loss: 1.6424
Epoch [1/100] Loss: 2.1009
Epoch [1/100] Loss: 1.7577
Epoch [1/100] Loss: 2.1110
Epoch [1/100] Loss: 1.8239
Epoch [1/100] Loss: 2.2993
Epoch [1/100] Loss: 1.7317
Epoch [1/100] Loss: 1.4285
Epoch [1/100] Loss: 1.6779
Epoch [1/100] Loss: 1.4715
Epoch [1/100] Loss: 1.5926
Epoch [1/100] Loss: 2.0866
Epoch [1/100] Loss: 1.4419
Epoch [1/100] Loss: 1.6893
Epoch [1/100] Loss: 1.5883
Epoch [1/100] Loss: 1.8504
Epoch [1/100] Loss: 2.0572
Epoch [1/100] Loss: 1.8225
Epoch [1/100] Loss: 1.6913
Epoch [1/100] Loss: 1.3329
E

1684it [09:01,  3.11it/s]
/home/dongwonshin/Desktop/pytorch_dataloader/ptsemseg/metrics.py:32: RuntimeWarning: invalid value encountered in true_divide
  acc_cls = np.diag(hist) / hist.sum(axis=1)
/home/dongwonshin/Desktop/pytorch_dataloader/ptsemseg/metrics.py:34: RuntimeWarning: invalid value encountered in true_divide
  iu = np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist))


Overall Acc: 	 0.497896578486
Mean Acc : 	 0.22475549179
FreqW Acc : 	 0.318164428397
Mean IoU : 	 0.15604718532
Epoch [2/100] Loss: 1.2851
Epoch [2/100] Loss: 1.6099
Epoch [2/100] Loss: 1.5434
Epoch [2/100] Loss: 1.4631
Epoch [2/100] Loss: 1.2725
Epoch [2/100] Loss: 1.4018
Epoch [2/100] Loss: 1.6308
Epoch [2/100] Loss: 1.3629
Epoch [2/100] Loss: 1.7571
Epoch [2/100] Loss: 1.4372
Epoch [2/100] Loss: 1.4311
Epoch [2/100] Loss: 1.4186
Epoch [2/100] Loss: 1.4404
Epoch [2/100] Loss: 1.0792
Epoch [2/100] Loss: 1.4540
Epoch [2/100] Loss: 1.4279
Epoch [2/100] Loss: 1.5042
Epoch [2/100] Loss: 1.3765
Epoch [2/100] Loss: 1.7781
Epoch [2/100] Loss: 1.3850
Epoch [2/100] Loss: 1.5592
Epoch [2/100] Loss: 1.2388
Epoch [2/100] Loss: 1.1379
Epoch [2/100] Loss: 1.3472
Epoch [2/100] Loss: 1.4526
Epoch [2/100] Loss: 1.4578
Epoch [2/100] Loss: 1.1748
Epoch [2/100] Loss: 1.3627
Epoch [2/100] Loss: 1.3082
Epoch [2/100] Loss: 1.4672
Epoch [2/100] Loss: 1.4100
Epoch [2/100] Loss: 1.5394
Epoch [2/100] Loss: 1.7